<div style="height:80px; background-color:#C5DABE; font-family:Arial; color:#4B296B; font-size:45px; font-weight:Bold; text-align:center; border:2px solid #618E52; border-radius:10px; padding-top:4px;">Movies Recommendation System</div>

<div style="text-align:center;"><img src="https://www.themoviedb.org/assets/2/v4/logos/v2/blue_short-8e7b30f73a4020692ccca9c88bafe5dcb6f8a62a4c6bc55cd9ba82bb2cd95f6c.svg" width="1000"></div>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Importing Required Libraries
# for Movies Poster Reuest
import requests
from IPython.display import Image, HTML, display

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import warnings
def warns(*args,**kwargs): pass
warnings.warn=warns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<div style="height:60px; font-family:Arial; color:#4B296B; font-size:30px; font-weight:Bold; text-align:center; border:2px solid #618E52; border-radius:10px; padding-top:8px;">Data Loading...</div>

In [ ]:
dfmovies = pd.read_csv("/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv")
dfcredits = pd.read_csv("/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv")
print("Movies Shape: ", dfmovies.shape)
print("Credits Shape: ", dfcredits.shape)

<div style="height:60px; font-family:Arial; color:#4B296B; font-size:30px; font-weight:Bold; text-align:center; border:2px solid #618E52; border-radius:10px; padding-top:6px;">Exploratory Data Analysis</div>

In [ ]:
dfmovies.head()

In [ ]:
dfcredits.head()

In [ ]:
print("-"*20, "Movies Columns", "-"*20, "\n", dfmovies.columns)
print("\n", "-"*20, "Credites Columns", "-"*20, "\n", dfcredits.columns)

<div style="height:60px; font-family:Arial; color:#4B296B; font-size:25px; font-weight:Bold; text-align:center; border:2px solid #618E52; border-radius:10px; padding-top:8px;">Top 5-Movies as per Popularity Calculation</div>

In [ ]:
popular_movies = dfmovies.nlargest(n=5, columns=['popularity'])[['id', 'title']]
getList_name = {}
for x, xRows in popular_movies.iterrows():
    #print(xRows['id'])
    getResponse = requests.get('https://api.themoviedb.org/3/movie/{}?api_key=c0bda0be71f7815fd6ba2eb5f5c86fd8'.format(xRows['id']) )
    getData = getResponse.json()
    getPath = "http://image.tmdb.org/t/p/w500" + getData['poster_path']
    getList_name[xRows['title']] = getPath

display( HTML(f"""<table>
                    <tr>
                        <td><img src={list(getList_name.values())[0]} style='border-radius:10px; height:300px; width:575px; border:1px solid #999;'></td>
                        <td><img src={list(getList_name.values())[1]} style='border-radius:10px; height:300px; width:575px; border:1px solid #999;'></td>
                        <td><img src={list(getList_name.values())[2]} style='border-radius:10px; height:300px; width:575px; border:1px solid #999;'></td>
                        <td><img src={list(getList_name.values())[3]} style='border-radius:10px; height:300px; width:575px; border:1px solid #999;'></td>
                        <td><img src={list(getList_name.values())[4]} style='border-radius:10px; height:300px; width:575px; border:1px solid #999;'></td>
                    </tr>
                    <tr>
                        <td><div style="height:40px; padding-top:15px; text-align:center; font-size:14px; font-weight:bold; border:1px solid #ccc; border-radius:10px;">{list(getList_name.keys())[0]}</div></td>
                        <td><div style="height:40px; padding-top:15px; text-align:center; font-size:14px; font-weight:bold; border:1px solid #ccc; border-radius:10px;">{list(getList_name.keys())[1]}</div></td>
                        <td><div style="height:40px; padding-top:15px; text-align:center; font-size:14px; font-weight:bold; border:1px solid #ccc; border-radius:10px;">{list(getList_name.keys())[2]}</div></td>
                        <td><div style="height:40px; padding-top:15px; text-align:center; font-size:14px; font-weight:bold; border:1px solid #ccc; border-radius:10px;">{list(getList_name.keys())[3]}</div></td>
                        <td><div style="height:40px; padding-top:15px; text-align:center; font-size:14px; font-weight:bold; border:1px solid #ccc; border-radius:10px;">{list(getList_name.keys())[4]}</div></td>
                    </tr>
                </table>"""))

<div style="height:60px; font-family:Arial; color:#4B296B; font-size:30px; font-weight:Bold; text-align:center; border:2px solid #618E52; border-radius:10px; padding-top:6px;">Data Merging</div>

In [ ]:
# Combine the 2 dataframes in single on title base
dfcombine = dfmovies.merge(dfcredits, on='title')
print("Cobine dataframe Shape: ", dfcombine.shape)

In [ ]:
dfcombine.columns

In [ ]:
# Re-Create the Dataset with the selected columns for recomender system
# ['id', 'title', 'original_title', 'tagline', 'genres', 'keywords', 'cast', 'crew', 'overview']

features_cols = ['id', 'title', 'original_title', 'tagline', 'genres', 'keywords', 'cast', 'crew', 'overview']

df_final = dfcombine[features_cols]
df_final.head()

<div style="height:60px; font-family:Arial; color:#4B296B; font-size:30px; font-weight:Bold; text-align:center; border:2px solid #618E52; border-radius:10px; padding-top:6px;">Missing Values</div>

In [ ]:
# Checking Missing Values
df_final.isnull().sum()

In [ ]:
#Verifying Missing Values
df_final['tagline'] = df_final['tagline'].fillna('unknown')
df_final.dropna(inplace=True)
df_final.isnull().sum()

In [ ]:
df_final.head()

<div style="height:60px; font-family:Arial; color:#4B296B; font-size:30px; font-weight:Bold; text-align:center; border:2px solid #618E52; border-radius:10px; padding-top:6px;">Data-Processing for Movie Tags</div>

In [ ]:
#Verify the Overview of Index 0
df_final['overview'][0]

In [ ]:
#Verify Overview Column by Applying Split Function
df_final['overview'].apply(lambda x: x.lower() and x.split() )

In [ ]:
# For Creating Tags, add new columns with name [tags]
#Apply Split Function on Overview Column and Save in New Columns [tags]
df_final['tags'] = df_final['overview'].apply(lambda x: x.lower() and x.split() )
df_final['tags'][0]

In [ ]:
# Verify the [genres] values
df_final.iloc[0]['genres']

In [ ]:
# Verify the [keywords] values
df_final.iloc[0]['keywords']

In [ ]:
# import module [ast]
# ast module have fuction [literal_eval] that converts the string in list
# Create function for [genres] and [keywords]

import ast
def colData_Transform(obj):
    lst = []
    for i in ast.literal_eval(obj):
        lst.append(i['name'].replace(" ","").lower())
    return lst

In [ ]:
# Apply Function 
df_final['tags'] = df_final['tags'] + df_final['genres'].apply(colData_Transform)
df_final['tags'] = df_final['tags'] + df_final['keywords'].apply(colData_Transform)
df_final['tags'][0]

In [ ]:
# Function to Extract the first 5-names from cast
# Create Function for [Cast]
def fetch_cast_data(obj):
    lst = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter < 5:
            lst.append(i['name'].replace(" ","").lower())
            counter += 1

    return lst

In [ ]:
df_final['tags'] = df_final['tags'] + df_final['cast'].apply(fetch_cast_data)
df_final['tags'][0]

In [ ]:
df_final['crew'][0]

In [ ]:
# Function to Extract the Director Name from Crew
# Create Function for [Crew]
def get_crew_data(obj):
    lst = []
    for x in ast.literal_eval(obj):
        if x['job'] == "Director":
            lst.append(x['name'].replace(" ","").lower())

    return lst

In [ ]:
df_final['tags'] = df_final['tags'] + df_final['crew'].apply(get_crew_data)
df_final['tags'][0]

In [ ]:
# Convert movie title in list and concatenate with tags
df_final['tags'] = df_final['tags'] + df_final['title'].apply(lambda x: x.lower() and x.split() )

In [ ]:
# Convert movie original_title in list and concatenate with tags
# if original_title not equal to title then concatenate otherwise skip

for x, xRow in df_final.iterrows():
    if (xRow['original_title'] != xRow['title'] ):
        df_final.at[x, 'tags'] = xRow['tags'] + xRow['original_title'].lower().split()
        
df_final['tags'][0]

In [ ]:
# Converting taglines in list for concatenation with tags
df_final['tagline'].apply(lambda x: x.lower() and x.split() )

In [ ]:
# Convert movie tagline in list and concatenate with tags
# if tagline 'unknown' then skip

for x, xRow in df_final.iterrows():
    if (xRow['tagline'] != 'unknown'):
        df_final.at[x, 'tags'] = xRow['tags'] + xRow['tagline'].lower().split()

df_final['tags'][0]

In [ ]:
df_final.columns

In [ ]:
# Create new DataFrame with required columns only
df_new_final = df_final[['id', 'title', 'tags']]
df_new_final.head()

In [ ]:
# Transform list of tags in the string and in lowercase
df_new_final['tags'] = df_new_final['tags'].apply(lambda x: " ".join(x).lower())
df_new_final['tags'] = df_new_final['tags'].str.lower()
df_new_final.head()

<div style="height:60px; font-family:Arial; color:#4B296B; font-size:30px; font-weight:Bold; text-align:center; border:2px solid #618E52; border-radius:10px; padding-top:6px;">Steeming</div>

In [ ]:
# Apply Steming to remove similarities/duplications in words list
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
# Apply steming on selected column and return in string
def xStem(txt):
    y = []
    for x in txt.split():
        y.append(ps.stem(x))
    return " ".join(y)

In [ ]:
df_new_final['tags'] = df_new_final['tags'].apply(xStem)
df_new_final['tags'][0]

<div style="height:60px; font-family:Arial; color:#4B296B; font-size:30px; font-weight:Bold; text-align:center; border:2px solid #618E52; border-radius:10px; padding-top:6px;">Vectorization</div>

In [ ]:
# Removing Stop-words
from sklearn.feature_extraction.text import CountVectorizer
countVec = CountVectorizer(max_features=10000, stop_words='english')

In [ ]:
# Tags of Movies Convert in Vectors
dataVectors = countVec.fit_transform(df_new_final['tags']).toarray()
dataVectors

In [ ]:
dataVectors[0]

In [ ]:
countVec.get_feature_names()[130:250]

<div style="height:60px; font-family:Arial; color:#4B296B; font-size:30px; font-weight:Bold; text-align:center; border:2px solid #618E52; border-radius:10px; padding-top:4px;">Calculate Vectors Similarities & Distances</div>

In [ ]:
# Import cosine_similarity to Calulate Distance between the Vectors
from sklearn.metrics.pairwise import cosine_similarity

# Calculate the Distance of all Movies with eachothers
similarity = cosine_similarity(dataVectors)

# Shape
similarity.shape

In [ ]:
# Get the Array of Arrays for all Movies
similarity

In [ ]:
# Similarity of 1st Movies with all
# Similarity with itself will be Index of 0

print("1st Sub Array Shape: ",similarity[0].shape)
print("\n", "-"*35, "\n Similarity of First Movie [Avatar] ","\n","-"*35, "\n", similarity[0])

In [ ]:
#Conver list of tuples and index created that help in sorting
sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x:x[1])[1:6] # from 1-5, 0 will be same movies

<div style="height:80px; background-color:#C5DABE; font-family:Arial; color:#4B296B; font-size:45px; font-weight:Bold; text-align:center; border:2px solid #618E52; border-radius:10px; padding-top:4px;">Recommendations</div>

In [ ]:
# Recommender Function to Return Movie Names Only
def recommend_movies_names(xMovie):
    # Get Index of given Movie
    movie_index = df_new_final[df_new_final["title"].str.lower() == xMovie.lower()].index[0]
    distances = similarity[movie_index]
    listofMovies = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]

    for i in listofMovies:
        print(df_new_final.iloc[i[0]]["title"])

In [ ]:
# Recommended Movies Name with Example of 3-Movies
movies_list = {"Avatar", "Batman", "Titanic"}
for x in movies_list:
    print("-"*20, "Recommendations for [", x,"]", "-"*20)
    recommend_movies_names(x)

<div style="height:60px; font-family:Arial; color:#4B296B; font-size:30px; font-weight:Bold; text-align:center; border:2px solid #618E52; border-radius:10px; padding-top:6px;">Recommend Movie Names with Poster</div>

In [ ]:
# Function that fetch Recommended Movies Poster Path
# with the help of API
def fetch_movies_poster(movieID):
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key=c0bda0be71f7815fd6ba2eb5f5c86fd8'.format(movieID) )
    data = response.json()
    posterPath = "http://image.tmdb.org/t/p/w500" + data['poster_path']
    return posterPath



# Movie Recomender Function, 
# Also call movies poster function 
# Return list of Recommended Movie Names and poster path
def getRecommended_movies_name(xMovie):
    # Get Index of given Movie
    movie_index = df_new_final[df_new_final["title"].str.lower() == xMovie.lower()].index[0]
    distances = similarity[movie_index]
    listofMovies = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]

    recomended_movies = []
    movies_poster = []
    for i in listofMovies:
        recomended_movies.append(df_new_final.iloc[i[0]]["title"])
        # API required to fetch the poster_path
        movies_poster.append( fetch_movies_poster(df_new_final.iloc[i[0]]["id"]) )
        
    return recomended_movies, movies_poster
    

In [ ]:
# Function that Display the Recommended Movie Names and Posters
def show_poster(sel_movie, mov_name, posterPath):
    display( HTML(f"""<div style="font-size:24px; font-weight:Bold; color:#fff; text-align:center; padding-top:8px; height:12%; width: 100%; border:1px solid #ccc; border-radius:10px; margin-top:10px; background-color:green;">{sel_movie}</div>
                   <table>
                    <tr>
                        <td><img src={posterPath[0]} style='border-radius:10px; width:575px; height:300px;'></td>
                        <td><img src={posterPath[1]} style='border-radius:10px; width:575px; height:300px;'></td>
                        <td><img src={posterPath[2]} style='border-radius:10px; width:575px; height:300px;'></td>
                        <td><img src={posterPath[3]} style='border-radius:10px; width:575px; height:300px;'></td>
                        <td><img src={posterPath[4]} style='border-radius:10px; width:575px; height:300px;'></td>
                    </tr>
                    <tr>
                        <td><p style="text-align:center; font-size:14px; font-weight:bold">{mov_name[0]}</p></td>
                        <td><p style="text-align:center; font-size:14px; font-weight:bold">{mov_name[1]}</p></td>
                        <td><p style="text-align:center; font-size:14px; font-weight:bold">{mov_name[2]}</p></td>
                        <td><p style="text-align:center; font-size:14px; font-weight:bold">{mov_name[3]}</p></td>
                        <td><p style="text-align:center; font-size:14px; font-weight:bold">{mov_name[4]}</p></td>
                    </tr>
                   </table>""") )

In [ ]:
# Show the Recommended Name and Poster of Movies in List
movies_list = {"Avatar", "Batman"}
for x in movies_list:
    names, poster = getRecommended_movies_name(x)
    show_poster(x, names, poster)
    


In [ ]:
# Show the Recommended Movies of Selected Movie Name [Titanic]
movie_name = "Titanic"
names, poster = getRecommended_movies_name(movie_name)

In [ ]:
# Code to Show the recommended movie names and poster
display( HTML(f"""<div style="font-size:24px; font-weight:Bold; color:#fff; text-align:center; padding-top:8px; height:12%; width: 100%; border:1px solid #ccc; border-radius:10px; margin-top:10px; background-color:green;">{movie_name}</div>
                  <table>
                    <tr>
                        <td><img src={poster[0]} style='border-radius:10px; width:575px; height:300px;'></td>
                        <td><img src={poster[1]} style='border-radius:10px; width:575px; height:300px;'></td>
                        <td><img src={poster[2]} style='border-radius:10px; width:575px; height:300px;'></td>
                        <td><img src={poster[3]} style='border-radius:10px; width:575px; height:300px;'></td>
                        <td><img src={poster[4]} style='border-radius:10px; width:575px; height:300px;'></td>
                    </tr>
                    <tr>
                        <td><p style="text-align:center; font-size:14px; font-weight:bold">{names[0]}</p></td>
                        <td><p style="text-align:center; font-size:14px; font-weight:bold">{names[1]}</p></td>
                        <td><p style="text-align:center; font-size:14px; font-weight:bold">{names[2]}</p></td>
                        <td><p style="text-align:center; font-size:14px; font-weight:bold">{names[3]}</p></td>
                        <td><p style="text-align:center; font-size:14px; font-weight:bold">{names[4]}</p></td>
                    </tr>
                   </table>""") )


# **Good Luck** 
<div style="text-align:center;"><img src="https://seeklogo.com/images/S/Smile-logo-A18DBD791A-seeklogo.com.png" width="40" height="40"></div>

# Your upvote <img src="http://s3.amazonaws.com/pix.iemoji.com/images/emoji/apple/ios-12/256/thumbs-up.png" width="40" height="40"> and comments helps to motivate

<div style="text-align:center;"><img src="https://www.themoviedb.org/assets/2/v4/logos/v2/blue_short-8e7b30f73a4020692ccca9c88bafe5dcb6f8a62a4c6bc55cd9ba82bb2cd95f6c.svg" width="500"></div>